<a href="https://colab.research.google.com/github/Judy-Choi/Tensorflow_Certificate/blob/main/16_power.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# val_mae = 0.05

In [ ]:
!pip uninstall tensorflow 
!pip install tensorflow==2.10.0

Found existing installation: tensorflow 2.11.0
Uninstalling tensorflow-2.11.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.9/dist-packages/tensorflow-2.11.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.11.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.7/489.7 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 KB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.9 MB/s eta 0:00:00
  Attempting 

# 정답

In [ ]:
# ABOUT THE DATASET
#
# Original Source:
# https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption
#
# The original 'Individual House Hold Electric Power Consumption Dataset'
# has Measurements of electric power consumption in one household with
# a one-minute sampling rate over a period of almost 4 years.
#
# Different electrical quantities and some sub-metering values are available.
#
# For the purpose of the examination we have provided a subset containing
# the data for the first 60 days in the dataset. We have also cleaned the
# dataset beforehand to remove missing values. The dataset is provided as a
# csv file in the project.
#
# The dataset has a total of 7 features ordered by time.
# ==============================================================================
#
# INSTRUCTIONS
#
# Complete the code in following functions:
# 1. windowed_dataset()
# 2. solution_model()
#
# The model input and output shapes must match the following
# specifications.
#
# 1. Model input_shape must be (BATCH_SIZE, N_PAST = 24, N_FEATURES = 7),
#    since the testing infrastructure expects a window of past N_PAST = 24
#    observations of the 7 features to predict the next 24 observations of
#    the same features.
#
# 2. Model output_shape must be (BATCH_SIZE, N_FUTURE = 24, N_FEATURES = 7)
#
# 3. DON'T change the values of the following constants
#    N_PAST, N_FUTURE, SHIFT in the windowed_dataset()
#    BATCH_SIZE in solution_model() (See code for additional note on
#    BATCH_SIZE).
# 4. Code for normalizing the data is provided - DON't change it.
#    Changing the normalizing code will affect your score.
#
# HINT: Your neural network must have a validation MAE of approximately 0.055 or
# less on the normalized validation dataset for top marks.
#
# WARNING: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure.
#
# WARNING: If you are using the GRU layer, it is advised not to use the
# 'recurrent_dropout' argument (you can alternatively set it to 0),
# since it has not been implemented in the cuDNN kernel and may
# result in much longer training times.

In [ ]:
import urllib
import os
import zipfile
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()
        
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=(n_past + n_future), shift = shift, drop_remainder = True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.shuffle(len(series))
    ds = ds.map(
        lambda w: (w[:n_past], w[n_past:])
    )
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    download_and_extract_data()
    
    df = pd.read_csv('household_power_consumption.csv', sep=',', infer_datetime_format=True, index_col='datetime', header=0)
    
    N_FEATURES = len(df.columns)
    
    data = df.values
    split_time = int(len(data) * 0.5)
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))
    
    x_train = data[:split_time]
    x_valid = data[split_time:]
    
    # DO NOT CHANGE THIS CODE
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    BATCH_SIZE = 32
    N_PAST = 24
    N_FUTURE = 24
    SHIFT = 1

    train_set = windowed_dataset(series=x_train, 
                                 batch_size=BATCH_SIZE,
                                 n_past=N_PAST, 
                                 n_future=N_FUTURE,
                                 shift=SHIFT)
    
    valid_set = windowed_dataset(series=x_valid, 
                                 batch_size=BATCH_SIZE,
                                 n_past=N_PAST, 
                                 n_future=N_FUTURE,
                                 shift=SHIFT)
    
    
    model = tf.keras.models.Sequential([
        Conv1D(filters=64, 
               kernel_size=5,
               padding="causal",
               activation="relu",
               input_shape=[N_PAST, 7],
               ),
        Bidirectional(LSTM(32, return_sequences=True)),
        Bidirectional(LSTM(32, return_sequences=True)), 
        Dense(64, activation="relu"),
        Dropout(0.5),     
        Dense(32, activation="relu"),   
        Dense(N_FEATURES)
    ])
    
    checkpoint_path='model/my_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_mae',
                             verbose=1,
                             )
    early_stopping = EarlyStopping(monitor='val_mae', patience=10)

    model.compile(loss='huber',
              optimizer='adam', 
              metrics=["mae"]
              )

    model.fit(train_set, 
          validation_data=(valid_set), 
          epochs=100, 
          callbacks=[checkpoint, early_stopping], 
          )
    model.load_weights(checkpoint_path)
    return model

if __name__ == '__main__':
    model = solution_model()
    model.save("power-1.h5")
## don't copy following codes in real test
    from google.colab import files
    files.download('power-1.h5')

Epoch 1/100
   1347/Unknown - 38s 13ms/step - loss: 0.0077 - mae: 0.0646
Epoch 00001: val_mae improved from inf to 0.06423, saving model to model/my_checkpoint.ckpt
1349/1349 [==============================] - 60s 29ms/step - loss: 0.0077 - mae: 0.0646 - val_loss: 0.0073 - val_mae: 0.0642
Epoch 2/100
1347/1349 [============================>.] - ETA: 0s - loss: 0.0065 - mae: 0.0553
Epoch 00002: val_mae improved from 0.06423 to 0.05133, saving model to model/my_checkpoint.ckpt
1349/1349 [==============================] - 41s 25ms/step - loss: 0.0065 - mae: 0.0553 - val_loss: 0.0060 - val_mae: 0.0513
Epoch 3/100
1349/1349 [==============================] - ETA: 0s - loss: 0.0062 - mae: 0.0530
Epoch 00003: val_mae did not improve from 0.05133
1349/1349 [==============================] - 40s 25ms/step - loss: 0.0062 - mae: 0.0530 - val_loss: 0.0061 - val_mae: 0.0538
Epoch 4/100
1349/1349 [==============================] - ETA: 0s - loss: 0.0060 - mae: 0.0515
Epoch 00004: val_mae improved fr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ChatGPT

Q) 다음 코드의 성능을 개선해 줘.
사용하는 데이터는 float 형의 시계열 데이터이고
다음 조건을 적용해 줘.
Use dropout
Use batch normalization
Use checkpoint with save best only, and save weight only and verbose = 1
Use early stopping with patience = 10
Use reduce learning rate on plateau
set verbose = 1
Set epochs = 100
do not use data augmentation
Use other methods to make val loss value lower.

In [3]:
import urllib
import os
import zipfile
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()
        
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=(n_past + n_future), shift = shift, drop_remainder = True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.shuffle(len(series))
    ds = ds.map(
        lambda w: (w[:n_past], w[n_past:])
    )
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    download_and_extract_data()
    
    df = pd.read_csv('household_power_consumption.csv', sep=',', infer_datetime_format=True, index_col='datetime', header=0)
    
    N_FEATURES = len(df.columns)
    
    data = df.values
    split_time = int(len(data) * 0.5)
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))
    
    x_train = data[:split_time]
    x_valid = data[split_time:]
    
    # DO NOT CHANGE THIS CODE
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    BATCH_SIZE = 32
    N_PAST = 24
    N_FUTURE = 24
    SHIFT = 1

    train_set = windowed_dataset(series=x_train, 
                                 batch_size=BATCH_SIZE,
                                 n_past=N_PAST, 
                                 n_future=N_FUTURE,
                                 shift=SHIFT)
    
    valid_set = windowed_dataset(series=x_valid, 
                                 batch_size=BATCH_SIZE,
                                 n_past=N_PAST, 
                                 n_future=N_FUTURE,
                                 shift=SHIFT)
    
    
    model = tf.keras.models.Sequential([
        Conv1D(filters=64,
            kernel_size=5,
            padding="causal",
            activation="relu",
            input_shape=[N_PAST, 7],
            ),
        BatchNormalization(),
        Bidirectional(LSTM(32, return_sequences=True)),
        BatchNormalization(),
        Bidirectional(LSTM(32, return_sequences=True)),
        BatchNormalization(),
        Dense(64, activation="relu"),
        Dropout(0.5),
        Dense(32, activation="relu"),
        Dense(N_FEATURES)
    ])

    checkpoint_path = 'model/my_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path,
                                save_weights_only=True,
                                save_best_only=True,
                                monitor='val_mae',
                                verbose=1,
                                )
    early_stopping = EarlyStopping(monitor='val_mae', patience=10)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)

    optimizer = tf.keras.optimizers.Adam(lr=0.001)

    model.compile(loss='huber',
                optimizer=optimizer,
                metrics=["mae"]
                )

    model.fit(train_set,
            validation_data=valid_set,
            epochs=100,
            callbacks=[checkpoint, early_stopping, reduce_lr],
            verbose=1,
            )
    
    model.load_weights(checkpoint_path)
    return model

if __name__ == '__main__':
    model = solution_model()
    model.save("power-1.h5")
## don't copy following codes in real test
    # from google.colab import files
    # files.download('power-1.h5')

Epoch 1/100
   1349/Unknown - 49s 17ms/step - loss: 0.0120 - mae: 0.0883
Epoch 1: val_mae improved from inf to 0.06085, saving model to model/my_checkpoint.ckpt
1349/1349 [==============================] - 65s 29ms/step - loss: 0.0120 - mae: 0.0883 - val_loss: 0.0071 - val_mae: 0.0609 - lr: 0.0010
Epoch 2/100
1349/1349 [==============================] - ETA: 0s - loss: 0.0071 - mae: 0.0633
Epoch 2: val_mae improved from 0.06085 to 0.05718, saving model to model/my_checkpoint.ckpt
1349/1349 [==============================] - 42s 26ms/step - loss: 0.0071 - mae: 0.0633 - val_loss: 0.0064 - val_mae: 0.0572 - lr: 0.0010
Epoch 3/100
1347/1349 [============================>.] - ETA: 0s - loss: 0.0066 - mae: 0.0578
Epoch 3: val_mae improved from 0.05718 to 0.05406, saving model to model/my_checkpoint.ckpt
1349/1349 [==============================] - 42s 26ms/step - loss: 0.0066 - mae: 0.0578 - val_loss: 0.0063 - val_mae: 0.0541 - lr: 0.0010
Epoch 4/100
1349/1349 [==============================

# 직접 코딩

과제 순서를 바꿔서, ChatGPT 가 짜 준 코드를 직접 개선해 본다

...

개선이 되지 않아 ChatGPT 에 추가 질문을 해 봄

Q) 시계열 데이터를 예측하는 문제에서 다음 모델 구조를 개선해 줘

A) 이 모델은 Conv1D 레이어를 사용하여 입력 시계열 데이터를 처리하고, 그 다음에 두 개의 Bidirectional LSTM 레이어를 쌓아서 시계열 데이터를 학습하고 예측합니다. 그리고 뒤이어 Dense 레이어들을 통해 출력을 수행합니다.

모델 구조 자체는 꽤 괜찮은 편이나 몇 가지 개선 가능한 점이 있습니다.

BatchNormalization 추가하기
배치 정규화는 각 레이어의 입력값을 평균과 분산으로 정규화하는 방법입니다. 이를 추가하면 학습이 안정화되고 더 빠르게 수렴하게 됩니다. 예측 결과도 개선될 가능성이 있습니다.

Residual Connection 추가하기
Residual Connection은 출력값과 입력값을 더하는 방식으로 네트워크를 구성하는 방법입니다. 이를 추가하면 학습이 더 안정화되고 더 깊은 모델을 학습할 수 있게 됩니다.

다른 활성화 함수 적용해보기
현재 모델에서는 ReLU 활성화 함수를 사용하고 있습니다. 이 외에도 다른 활성화 함수를 적용하여 성능 개선을 시도할 수 있습니다. 예를 들어, tanh, sigmoid 등의 함수를 사용해보는 것도 좋은 방법입니다.

다양한 하이퍼파라미터 조합 시도하기
학습률, 배치 크기, 에포크 수 등 다양한 하이퍼파라미터를 조합하여 실험해보는 것도 모델 개선에 도움이 됩니다.

다른 모델 구조 시도하기
컨볼루션 신경망(Convolutional Neural Network, CNN)이나 Transformer 등의 다른 모델 구조를 시도해보는 것도 좋은 방법입니다. 다양한 모델을 실험하여 최적의 모델을 찾아내는 것이 중요합니다.

In [6]:
import urllib
import os
import zipfile
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()
        
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=(n_past + n_future), shift = shift, drop_remainder = True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.shuffle(len(series))
    ds = ds.map(
        lambda w: (w[:n_past], w[n_past:])
    )
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    download_and_extract_data()
    
    df = pd.read_csv('household_power_consumption.csv', sep=',', infer_datetime_format=True, index_col='datetime', header=0)
    
    N_FEATURES = len(df.columns)
    
    data = df.values
    split_time = int(len(data) * 0.5)
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))
    
    x_train = data[:split_time]
    x_valid = data[split_time:]
    
    # DO NOT CHANGE THIS CODE
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    BATCH_SIZE = 32
    N_PAST = 24
    N_FUTURE = 24
    SHIFT = 1

    train_set = windowed_dataset(series=x_train, 
                                 batch_size=BATCH_SIZE,
                                 n_past=N_PAST, 
                                 n_future=N_FUTURE,
                                 shift=SHIFT)
    
    valid_set = windowed_dataset(series=x_valid, 
                                 batch_size=BATCH_SIZE,
                                 n_past=N_PAST, 
                                 n_future=N_FUTURE,
                                 shift=SHIFT)
    
    model = tf.keras.models.Sequential([
        Conv1D(filters=64,
            kernel_size=5,
            padding="causal",
            activation="relu",
            input_shape=[N_PAST, 7],
            ),
        # BatchNormalization(),
        Bidirectional(LSTM(32, return_sequences=True)),
        # BatchNormalization(),
        Bidirectional(LSTM(32, return_sequences=True)),
        # BatchNormalization(),
        Dense(64, activation="relu"),
        Dropout(0.5),
        Dense(32, activation="relu"),
        Dense(N_FEATURES)
    ])

    checkpoint_path = 'model/my_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path,
                                save_weights_only=True,
                                save_best_only=True,
                                monitor='val_mae',
                                verbose=1,
                                )
    early_stopping = EarlyStopping(monitor='val_mae', patience=10)
    reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.1, patience=5, verbose=1)

    # optimizer SGD 로 교체 시 수렴되기까지 더 많은 epoch.
    # optimizer = tf.keras.optimizers.Adam(lr=0.001)
    optimizer = tf.keras.optimizers.SGD(lr=0.001)

    model.compile(loss='huber',
                optimizer=optimizer,
                metrics=["mae"]
                )

    model.fit(train_set,
            validation_data=valid_set,
            epochs=100,
            callbacks=[checkpoint, early_stopping, reduce_lr],
            verbose=1,
            )
    
    model.load_weights(checkpoint_path)
    return model

if __name__ == '__main__':
    model = solution_model()
    model.save("power-1.h5")
## don't copy following codes in real test
    # from google.colab import files
    # files.download('power-1.h5')

Epoch 1/100
   1348/Unknown - 31s 13ms/step - loss: 0.0185 - mae: 0.1206
Epoch 1: val_mae improved from inf to 0.10722, saving model to model/my_checkpoint.ckpt
1349/1349 [==============================] - 53s 29ms/step - loss: 0.0185 - mae: 0.1206 - val_loss: 0.0142 - val_mae: 0.1072 - lr: 0.0100
Epoch 2/100
1348/1349 [============================>.] - ETA: 0s - loss: 0.0143 - mae: 0.1118
Epoch 2: val_mae improved from 0.10722 to 0.10004, saving model to model/my_checkpoint.ckpt
1349/1349 [==============================] - 36s 23ms/step - loss: 0.0143 - mae: 0.1118 - val_loss: 0.0122 - val_mae: 0.1000 - lr: 0.0100
Epoch 3/100
1347/1349 [============================>.] - ETA: 0s - loss: 0.0132 - mae: 0.1065
Epoch 3: val_mae improved from 0.10004 to 0.09454, saving model to model/my_checkpoint.ckpt
1349/1349 [==============================] - 43s 28ms/step - loss: 0.0132 - mae: 0.1065 - val_loss: 0.0113 - val_mae: 0.0945 - lr: 0.0100
Epoch 4/100
1348/1349 [============================>.

KeyboardInterrupt: ignored